In [1]:
!pip3 install langchain_community


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.llms import Ollama

In [3]:
llm = Ollama(model = "llama3")
llm.invoke("the first man on the moon was....")

'The answer is: Neil Armstrong!\n\nOn July 20, 1969, NASA\'s Apollo 11 mission made history when Neil Armstrong became the first person to set foot on the Moon. He famously declared, "That\'s one small step for man, one giant leap for mankind," as he stepped off the lunar module Eagle and onto the Moon\'s surface.\n\nArmstrong was followed by fellow astronaut Edwin "Buzz" Aldrin, who also walked on the Moon during the mission. Michael Collins remained in orbit around the Moon in the command module Columbia.\n\nWhat an incredible achievement!'

In [4]:
# Read the trasaction 2022_2023.csv
import pandas as pd
df = pd.read_csv("transactions_2022_2023.csv") 
df

,Date,Name / Description,Expense/Income,Amount (EUR)
0,2023-12-30,Belastingdienst,Expense,9.96
1,2023-12-30,Tesco Breda,Expense,17.53
2,2023-12-30,Monthly Appartment Rent,Expense,451.00
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46
4,2023-12-29,Selling Paintings,Income,13.63
...,...,...,...,...
207,2023-10-14,Bookstore Purchase,Expense,31.50
208,2023-10-13,Online Shopping,Expense,68.50
209,2023-10-12,Car Insurance Premium,Expense,130.00
210,2023-10-11,Salary,Income,1850.00


In [5]:
unique_transactions = df["Name / Description"].unique()
len(unique_transactions) 

71

In [6]:
unique_transactions[1:10]

array(['Tesco Breda', 'Monthly Appartment Rent',
       'Vishandel Sier Amsterdam', 'Selling Paintings',
       'Spotify Ab By Adyen', 'Tk Maxx Amsterdam Da', 'Consulting',
       'Aidsfonds', 'Tls Bv Inz Ov-Chipkaart'], dtype=object)

In [7]:
response = llm.invoke("Can you add appropriate category next to each of the following expenese. Respong with a list o categories entertianment, Beta boulders ams ansterdam nld- Sports,  etc.: "+" ,".join(unique_transactions[1:30]))
response = response

In [8]:
response

"Here is the list with categories added:\n\n1. Tesco Breda - Groceries\n2. Monthly Apartment Rent - Housing\n3. Vishandel Sier Amsterdam - Home Improvement\n4. Selling Paintings - Income\n5. Spotify Ab By Adyen - Entertainment (Music)\n6. Tk Maxx Amsterdam Da - Retail Therapy\n7. Consulting - Work\n8. Aidsfonds - Charity/Donation\n9. Tls Bv Inz Ov-Chipkaart - Transportation (Public Transit)\n10. Etos Amsterdam - Health and Wellness\n11. Beta Boulders Ams Amsterdam - Sports (Climbing)\n12. Birtat Restaurant Amsterdam - Dining Out\n13. Freelancing - Work\n14. Tikkie - Bills Payment\n15. Blogging - Income\n16. Taxi Utrecht - Transportation (Taxis)\n17. Apple Services - Technology (Software)\n18. Amazon Lux - E-commerce\n19. Classpass* Monthly - Fitness/Gym\n20. Audible Uk AdblCo/Pymt Gbr - Entertainment (Audiobooks)\n21. Travel Expenses - Travel\n22. Monthly Phone Bill - Communication (Phone)\n23. Car Maintenance - Transportation (Vehicle)\n24. Medical Check-up - Health and Wellness\n25. 

In [9]:
#get index list

In [11]:
def hop(start, stop, step):
    for i in range(start, stop, step):
        yield i 
    yield stop 
    
index_list = list(hop(0, len(unique_transactions),10))
index_list

[0, 10, 20, 30, 40, 50, 60, 70, 71]

In [12]:
def categorize_transactions(transaction_names, llm):
    response = llm.invoke("Can you add an appropriate category to the following expenses. For example: Spotify AB by Adyen - Entertainment, Beta Boulders Ams Amsterdam Nld - Sport, etc.. Categories should be less than 4 words. " + transaction_names)
    response = response.split('\n')

    print(response)

    # Put in dataframe
    categories_df = pd.DataFrame({'Transaction vs category': response})
    categories_df[['Transaction', 'Category']] = categories_df['Transaction vs category'].str.split(' - ', expand=True)
    
    return categories_df

In [13]:

# Test out the function
categorize_transactions('ISS Catering Services De Meern, Taxi Utrecht, Etos AMSTERDAM NLD, Bistro Bar Amsterdam', llm)

['Here are the categorized expenses:', '', '1. Spotify AB by Adyen - **Entertainment**', '2. Beta Boulders Ams Amsterdam Nld - **Sport**', '3. ISS Catering Services De Meern - **Food/Work**', '4. Taxi Utrecht - **Transportation**', '5. Etos AMSTERDAM NLD - **Health/Beauty**', '6. Bistro Bar Amsterdam - **Entertainment**', '', "Let me know if you'd like me to add any other categories!"]


,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,1. Spotify AB by Adyen - **Entertainment**,1. Spotify AB by Adyen,**Entertainment**
3,2. Beta Boulders Ams Amsterdam Nld - **Sport**,2. Beta Boulders Ams Amsterdam Nld,**Sport**
4,3. ISS Catering Services De Meern - **Food/Work**,3. ISS Catering Services De Meern,**Food/Work**
5,4. Taxi Utrecht - **Transportation**,4. Taxi Utrecht,**Transportation**
6,5. Etos AMSTERDAM NLD - **Health/Beauty**,5. Etos AMSTERDAM NLD,**Health/Beauty**
7,6. Bistro Bar Amsterdam - **Entertainment**,6. Bistro Bar Amsterdam,**Entertainment**
8,,,None
9,Let me know if you'd like me to add any other ...,Let me know if you'd like me to add any other ...,None


In [14]:
categories_df_all = pd.DataFrame()

# Loop through the index_list
for i in range(0, len(index_list)-1):
    transaction_names = unique_transactions[index_list[i]:index_list[i+1]]
    transaction_names = ','.join(transaction_names)

    categories_df = categorize_transactions(transaction_names, llm)
    categories_df_all = pd.concat([categories_df_all, categories_df], ignore_index=True)

['Here are the categorized expenses:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam Nld - Sport', '3. Belastingdienst - Taxes', '4. Tesco Breda - Grocery', '5. Monthly Appartment Rent - Housing', '6. Vishandel Sier Amsterdam - Food', '7. Selling Paintings - Miscellaneous', '8. Tk Maxx Amsterdam Da - Shopping', '9. Consulting - Professional Services', '10. Aidsfonds - Charity', '11. Tls BV Inz Ov-Chipkaart - Transportation', '', 'Let me know if you need any further assistance!']
['Here are the categorized expenses:', '', '* Spotify AB by Adyen - Entertainment', '* Beta Boulders Ams Amsterdam Nld - Sport', '* Etos Amsterdam - Health', '* Bouldermuur Bv Amsterdam - Sport', '* Biratat Restaurant Amsterdam - Dining', '* Freelancing - Work', '* Tikkie - Bills', '* Blogging - Work', '* Taxi Utrecht - Transportation', '* Apple Services - Tech']
['Here are the categorized expenses:', '', '1. Spotify AB by Adyen - Entertainment', '2. Beta Boulders Ams Amsterdam N

In [15]:
categories_df_all

,Transaction vs category,Transaction,Category
0,Here are the categorized expenses:,Here are the categorized expenses:,None
1,,,None
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
...,...,...,...
110,* Spotify AB by Adyen - Entertainment,* Spotify AB by Adyen,Entertainment
111,* Beta Boulders Ams Amsterdam Nld - Sport,* Beta Boulders Ams Amsterdam Nld,Sport
112,* [Your expense] - [Category],* [Your expense],[Category]
113,,,None


In [16]:
# Get unique categories in categories_df_all
unique_categories = categories_df_all["Category"].unique()
unique_categories

array([None, 'Entertainment', 'Sport', 'Taxes', 'Grocery', 'Housing',
       'Food', 'Miscellaneous', 'Shopping', 'Professional Services',
       'Charity', 'Transportation', 'Health', 'Dining', 'Work', 'Bills',
       'Tech', 'Online Shopping', 'Fitness', 'Communication', 'Vehicle',
       'Education', 'Insurance', 'Books', '**Entertainment**',
       '**Sport**', '**Work Expenses**', '**Household**', '**Fitness**',
       '**Social**', '**Travel**', '**IT Services**', '**Food Out**',
       '**Social** (assuming this is not just a one-time lunch)',
       '**Hobbies**', '**Fashion**', '**Transportation**',
       '**Tech Subscriptions**', '**Home Maintenance**',
       '**Food Expenses**', '**Software Fees**', '**Gaming Subs**',
       '**Bills**', '**Work Income**', '**Healthcare**', 'Travel', 'Auto',
       'Internet', 'Retail', 'Utilities', 'Celebration', 'Services',
       '[Category]'], dtype=object)

In [20]:
# Drop NA values
categories_df_all = categories_df_all.dropna()

# If category contains "Food", then categorise as "Food and Drinks"
categories_df_all.loc[categories_df_all['Category'].str.contains("Food"), 'Category'] = "Food and Drinks"
# If category contains "Clothing", then categorise as "Clothing"
categories_df_all.loc[categories_df_all['Category'].str.contains("Clothing"), 'Category'] = "Clothing"
# If category contains "Services", then categorise as "Services"
categories_df_all.loc[categories_df_all['Category'].str.contains("Services"), 'Category'] = "Services"
# If category contains "Health" or "Wellness", then categorise as "Health and Wellness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Health|Wellness"), 'Category'] = "Health and Wellness"
# If category contains "Sport", then categorise as "Sport
#  and Fitness"
categories_df_all.loc[categories_df_all['Category'].str.contains("Sport"), 'Category'] = "Sport and Fitness"
# If category contains "Travel", then categorise as "Travel"
categories_df_all.loc[categories_df_all['Category'].str.contains("Travel"), 'Category'] = "Travel"

In [21]:
categories_df_all['Transaction'] = categories_df_all['Transaction'].str.replace(r'\d+\.\s+', '')
categories_df_all

,Transaction vs category,Transaction,Category
2,1. Spotify AB by Adyen - Entertainment,1. Spotify AB by Adyen,Entertainment
3,2. Beta Boulders Ams Amsterdam Nld - Sport,2. Beta Boulders Ams Amsterdam Nld,Sport and Fitness
4,3. Belastingdienst - Taxes,3. Belastingdienst,Taxes
5,4. Tesco Breda - Grocery,4. Tesco Breda,Grocery
6,5. Monthly Appartment Rent - Housing,5. Monthly Appartment Rent,Housing
...,...,...,...
104,11. Home Insurance Renewal - Insurance,11. Home Insurance Renewal,Insurance
105,12. Christmas Dinner - Celebration,12. Christmas Dinner,Celebration
110,* Spotify AB by Adyen - Entertainment,* Spotify AB by Adyen,Entertainment
111,* Beta Boulders Ams Amsterdam Nld - Sport,* Beta Boulders Ams Amsterdam Nld,Sport and Fitness


In [22]:
# Merge the categories_df_all with the transactions_2022_2023.csv dataframe (df)
df = pd.read_csv("transactions_2022_2023.csv")
df.loc[df['Name / Description'].str.contains("Spotify"), 'Name / Description'] = "Spotify Ab By Adyen"
df = pd.merge(df, categories_df_all, left_on='Name / Description', right_on='Transaction', how='left')
df

,Date,Name / Description,Expense/Income,Amount (EUR),Transaction vs category,Transaction,Category
0,2023-12-30,Belastingdienst,Expense,9.96,NaN,NaN,NaN
1,2023-12-30,Tesco Breda,Expense,17.53,NaN,NaN,NaN
2,2023-12-30,Monthly Appartment Rent,Expense,451.00,NaN,NaN,NaN
3,2023-12-30,Vishandel Sier Amsterdam,Expense,12.46,NaN,NaN,NaN
4,2023-12-29,Selling Paintings,Income,13.63,NaN,NaN,NaN
...,...,...,...,...,...,...,...
207,2023-10-14,Bookstore Purchase,Expense,31.50,NaN,NaN,NaN
208,2023-10-13,Online Shopping,Expense,68.50,NaN,NaN,NaN
209,2023-10-12,Car Insurance Premium,Expense,130.00,NaN,NaN,NaN
210,2023-10-11,Salary,Income,1850.00,NaN,NaN,NaN


In [23]:
df.to_csv("transactions_2022_2023_categorized.csv", index=False)